David Fleming Jan. 28,2015
Ipython notebook for working with the evolution of a circumbinary disk.

Case: Kepler 38 modelled after the initial conditions of Kley+2014

General consideration: Disk initialized as if it sees star at origin (COM) with total mass of the binary system.
Binary stars then replace single star at origin with positions/velocities such that COM coincides with the origin and the binary system starts out at the periastron.

Initial CB disk conditions:

$\Sigma \sim r^{-1/2}$

$T \sim r^{-1}$

$T_0 = 750K \ at \ r_0 = 1au$ corresponding to typical values for minimum mass solar nebula

$m_{disk} = 0.00383643566693 M_{\odot}$

$\mu = 2.35$

$Q_{min} = 1$

$R_{disk} = R_{max} = 2.0 au$

$R_{in} = 0.25 au$

cutlength = 0.01

$n_{gas} = 30000, n_{points} = 500 $

Initial Binary conditions:

$mass_{primary} = 0.949M_{\odot}$

$mass_{secondary} = 0.249M_{\odot}$

$a = 0.1469 au, \ e = 0.1032$

ChaNGa ICs:

dDelta = 0.0021 (dDelta > 0.0023 found to be unstable...binary system flew off screen)

dEta = 0.1

dTheta = 0.65

nSteps = 45000

For the rest, see the .param file

Contains basic code for anaylsis, plotting to be used later with my disks.

In [1]:
%matplotlib inline
#Imports required to run

import pynbody
from matplotlib.pylab import *
import matplotlib.pylab as plt
import pynbody.plot.sph as sph
import numpy as np

#Imports from ICgen-Binary directory
import sys
sys.path.append('/astro/users/dflemin3/Desktop/ICgen-Binary')
sys.path.append('/astro/users/dflemin3/Desktop')
import isaac
import AddBinary
import binaryUtils

#Typical plot parameters that make for pretty plots
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 18

/astro/users/dflemin3/Desktop/ICgen-Binary/isaac.pyc


In [2]:
#Load Snapshot containing final conditions
#Remember to have snapshot AND param file in same directory as this notebook
s = pynbody.load('k38_snap.std')

#Load initial conditions
s1 = pynbody.load('k38_snap.std')

IOError: File 'k38_snap.std': format not understood or does not exist

In [ ]:
#Make sure I have 2 stars in there with correct pos/vel
print s1.stars[0]['pos']
print s1.stars[1]['pos']

print s1.stars[0]['vel']
print s1.stars[1]['vel']

print s1.stars[0]['mass']
print s1.stars[1]['mass']

print
#Make sure I have 2 stars in there with correct pos/vel:
print s.stars[0]['pos']
print s.stars[1]['pos']

print s.stars[0]['vel']
print s.stars[1]['vel']

print s.stars[0]['mass']
print s.stars[1]['mass']

#Print current units
print s.stars[0]['pos'].units
print s.stars[0]['vel'].units

In [ ]:
print np.sum(s.gas['mass'])
print s.gas['mass'].units

In [ ]:
#Check CoM
"""
x1 = s1.stars[0]['pos']
x2 = s1.stars[1]['pos']
m1 = s1.stars[0]['mass']
m2 = s1.stars[1]['mass']

print AddBinary.calcCOM(m1,m2,x1,x2)
"""

x1 = s.stars[0]['pos']
x2 = s.stars[1]['pos']
m1 = s.stars[0]['mass']
m2 = s.stars[1]['mass']

print AddBinary.calcCOM(m1,m2,x1,x2)

In [ ]:
"""
Calculate changes in eccentricity, semimajor axis, stellar mass, and gas particle number over course of simulation.
"""

#Calculate Initial system parameters
x1 = s1.stars[0]['pos']
x2 = s1.stars[1]['pos']
v1 = s1.stars[0]['vel']
v2 = s1.stars[1]['vel']
m1 = s1.stars[0]['mass']
m2 = s1.stars[1]['mass']

ii = AddBinary.calcInc(x1, x2, v1, v2)

ei = AddBinary.calcEcc(x1, x2, v1, v2, m1, m2)

ai = AddBinary.calcSemi(x1, x2, v1, v2, m1, m2)
gi = len(s1.g)
mi = s1.stars['mass']

#Calculate final system parameters
x1 = s.stars[0]['pos']
x2 = s.stars[1]['pos']
v1 = s.stars[0]['vel']
v2 = s.stars[1]['vel']
m1 = s.stars[0]['mass']
m2 = s.stars[1]['mass']

ef = AddBinary.calcEcc(x1, x2, v1, v2, m1, m2)
af = AddBinary.calcSemi(x1, x2, v1, v2, m1, m2)
ifin = AddBinary.calcInc(x1, x2, v1, v2)
gf = len(s.g)
mf = s.stars['mass']

#Change in eccentricity
print "Initial eccentricity =",ei
print "Final eccentricity =",ef
print "Percent change in eccentricity = ",((ef-ei)/ei)*100
print ef-ei
print

#Change in semimajor axis
print "Initial semimajor axis =",ai
print "Final semimajor axis =",af
print "Percent change in semimajor axis = ",((af-ai)/ai)*100
print

#Change in inclination
print "Initial Inclination =", ii
print "Final Inclination =", ifin
print "Change in inclination = ",((ifin-ii))
print 

#Change in mass and # of gas particles eaten
print "Initial mass =",mi
print "Initial gas particles =",gi
print "Final mass =",mf
print "Final gas particles =",gf
print "Percent change in mass=",((mf-mi)/mi)*100
print "Change in gas particles =",(gf-gi)
print

In [ ]:
"""
#Calculate total angular momentum added by accreted gas
accretion =  binaryUtils.changaFloatSearch("temp.txt")

x1 = s1.stars[0]['pos']
x2 = s1.stars[1]['pos']
v1 = s1.stars[0]['vel']
v2 = s1.stars[1]['vel']
m1 = s1.stars[0]['mass']
m2 = s1.stars[1]['mass']

ac, ec = binaryUtils.linearMomentumEffects(x1, x2, v1, v2, m1, m2, accretion)

print af-ac
print ef-ec
print
print ac
print ec
"""

#Find inner disk edge
Following the prescription of Artymowics+1994, define the gap radius for a circumbinary disk to be at the disk edge position where the surface density is half of the maximum value in the disk.  In that paper, the gap size was studied for various binary eccentricities and mass ratios.  For Kepler 38, the mass ratio is 0.263 ~ 0.3 and the eccentricity is ~0.1.  These values correspond to a gap size of $r = 2.35 \pm 0.15$ times the binary semimajor axis.  For Kepler 38 whose a = 0.1469, one would expect to find after ~ 100 binary orbits a gap radius of ~0.34 au.  This gap formation occurs due to the tidal action of the binary system on the disk (and accretion where applicable).

The initial conditions of Artymowics+1994:

N_gas: ~ $10^4$
Disk Properties: axisymmetric; initialized with Keplerian velocities; $\Sigma \sim 1/r$; introduced binary into initially smooth disk (similar to my initialization).  For a binary with mass parameter $\mu = 0.3$ and eccentricity = 0.1, they found a gap at $r \approx 2.3a$ which is outside the 3:1 orbital period commensurability (Arty+1994 pg. 662).  


Note: In Kley+2014, they found that after ~2000yr of evolution, the inner edge of the disk to be $r \approx 0.45 au$ which corresponds to roughly $3a_{bin}$ which they claim is in good agreement with the results of Artymowics+1994.  The problem here is that Kley+2014 studied Kepler 38 where the mass ratio is about 0.3 and the eccentricity about 0.1.  The result that $r_{gap} \approx 3a_{bin}$ corresponds to a mass ratio of 0.3 and an eccentricity of 0.5 which does not hold for Kepler 38.


Below, I bin the disk surface density into radial bins and compute the radius at which the half maximum surface density occurs as described above.

In [ ]:
#Create profile.  Note: Choose nbins to be the same number of n points ICgen-Binary used to initial generate disk
p = pynbody.analysis.profile.Profile(s,max='10 au',min='0.01 au',type='log',nbins=5000)
pg = pynbody.analysis.profile.Profile(s.gas,nbins=5000)

#Find gap radius.  Defined in Artimowics+1994 where surface density is 1/2 maximum
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

#Find location of surface density maximum and look at radius r < r_max to find radius where surface density = max/2.0
value = pg['density'].max()/2.0
max_index = pg['density'].argmax()
print "Half Max Surface Density = {0} {1}".format(value,pg['density'].units)

#Look for inner gap edge from maximum value inwards
mask = (pg['rbins'] < pg['rbins'][max_index])
rho = pg['density'][mask]
i = find_nearest(rho,value)
print 'radius = {0}'.format(pg['rbins'][i])
print 'density at radius_max_1/2 = {0} {1}'.format(pg['density'][i],pg['density'].units)

In [ ]:
#Try plotting stuff
#Plot the gas surface density as a function of radius for test disk

#Now do the plotting with data derived from gas profile, pg
#In this case, tangential velocity vs. radius
plt.plot(pg['rbins'], pg['density'],'k', label = '$Gas$',)
plt.legend()
plt.xlabel("$R$ [AU]")
plt.ylabel(r'$\Sigma$ [M$_{\odot}$ au$^{-2}$]')
plt.yscale('log')
plt.xscale('log') #confirm -1/2 radius powerlaw relation
plt.xlim([0,5])
plt.axvline(x=0.22, ymin=-0.25, ymax=1, linewidth=2, color = 'r')

plt.show()
#Print units to confirm
print pg['rbins'].units
print pg['density'].units

In [ ]:
#Try plotting temperature profile of gas in plane of disk
#T vs. radius
plt.plot(pg['rbins'], pg['temp'],'k', label = '$Gas$')
plt.legend()
plt.loglog()
plt.xlabel("$R$ [AU]")
plt.ylabel("$T$ [K]")
plt.show()

In [ ]:
#Create nice color plot of temp as function of radius

#Not working for now since need to run image through changa once to get velocities
pynbody.analysis.angmom.faceon(s)
pynbody.analysis.halo.center(s,mode='hyb')
#pynbody.analysis.angmom.faceon(s1.g)#create an image of gas density integrated down the line of site (z axis) of width 50 Au
sph.image(s.g,qty="rho",units="g cm^-2",width=1,cmap="jet")

#Plot circle representing theoretical location of gap...should occur ~ 100 orbits
list_radians = [0]
 
# from degrees to radians, the 0 is already included so
# we don't make the universe collapse by dividing by zero.
for i in range(0,360):
    float_div = 180.0/(i+1)
    list_radians.append(pi/float_div)
     
# list of coordinates for each point
list_x2_axis = []
list_y2_axis = []

r = 2.3*ai #Inner gap edge predicted by Artimowicz+1994 for binary similar to Kepler 38
#r = 0.25
# calculate coordinates
# and append to above list
for a in list_radians:
    list_x2_axis.append(r*cos(a))
    list_y2_axis.append(r*sin(a))
 
# plot the coordinates
plt.plot(list_x2_axis,list_y2_axis,c='r')
plt.xlim([-0.5,0.5])
plt.ylim([-0.5,0.5])

In [ ]:
#Create verticle profiles

#Create verticle gas profile object
p_vert = pynbody.analysis.profile.VerticalProfile(s.gas, '0 au', '20 au', '25 au') #(range_min,range_max,max)

#Now plot density vs distance
#Confirm density units are correct
p_vert['density'].convert_units('Msol au**-3')
print p_vert['density'].units
print p_vert['rbins'].units
plt.plot(p_vert['rbins'], p_vert['density'],'k', hold=False)
plt.xlabel('$z$ [Au]')
plt.ylabel(r'$\rho_{\star}$ [M$_{\odot}$ Au$^{-3}$]')
plt.yscale('log')

In [ ]:
#Plot temperature vs. distance from plane
#Looks terrible since data is for small number of particles
plt.plot(p_vert['rbins'], p_vert['temp'],'k', hold=False)
plt.xlabel('$z$ [Au]'); plt.ylabel('$T$ [K]')

In [ ]:
#Plot velocity vectors
f, axs = plt.subplots(1,1,figsize=(8,8))

#create a simple slice showing the gas temperature, with velocity vectors overlaid
sph.velocity_image(s.g, vector_color="cyan", qty="temp",width=10,cmap="YlOrRd",
                   denoise=True,approximate_fast=False, show_cbar = False)

#plt.savefig("vectors.pdf")